# SESSIONS TRAIN

In [1]:
import pandas as pd, numpy as np
from tqdm.notebook import tqdm
import os, sys, pickle, glob, gc
from collections import Counter
import itertools
pd.set_option('display.max_columns', None)
from otto_utils import *

In [2]:
MODE = 0

In [3]:
type_labels = {'clicks':0, 'carts':1, 'orders':2}

if MODE==1:
    dataset = "otto-chunk-data-inparquet-format"
if MODE==0:
    dataset = "otto-validation"

def load_test():    
    dfs = []
    for e, chunk_file in enumerate(glob.glob(f'input/{dataset}/test_parquet/*')):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True)

test_df = load_test()
# test_df = pd.read_parquet("input/otto-shimacos-validation/shimacos_test_0.parquet")
print('Test data has shape',test_df.shape)
print(f'{len(set(test_df.session))} unique sessions')

Test data has shape (7683577, 4)
1801251 unique sessions


In [4]:
test_df = test_df.sort_values(['session','ts'],ascending=[True,True]).reset_index(drop=True)
session_features = test_df[['session']].drop_duplicates().reset_index(drop=True)

In [5]:
def add_new_col(session_features, fromdf, new_col, group_col, fillna=0, dtype='float'):
    session_features[new_col] = session_features[
        'session'
    ].map(
        dict(zip(
            fromdf['session'],
            fromdf[group_col]
        ))
    ).fillna(fillna).astype(dtype)

In [6]:
items = test_df.groupby(['session']).agg({'aid':['count','nunique']}).reset_index()
items.columns = ['session','actions','items']

for new_col, group_col in zip(
    ['session_actions','session_items'],
    ['actions','items']
):
    add_new_col(session_features, items, new_col, group_col, fillna=0, dtype='int16')

del items
gc_clear()

In [7]:
items_by_action = test_df.groupby(['session','type']).agg({'aid':['count','nunique']}).reset_index()
items_by_action.columns = ['session','type','actions','items']

for type_name, type_code in type_labels.items():
    add_new_col(
        session_features, 
        items_by_action[items_by_action['type']==type_code][['session','actions']], 
        new_col = f'session_{type_name}', 
        group_col = 'actions', 
        fillna = 0, 
        dtype = 'int16'
    )
    add_new_col(
        session_features, 
        items_by_action[items_by_action['type']==type_code][['session','items']], 
        new_col = f'session_items_{type_name[:-1]}ed', 
        group_col = 'items', 
        fillna = 0, 
        dtype = 'int16'
    )
    
del items_by_action
gc_clear()

In [8]:
sessions_length = test_df.groupby(['session'])['ts'].agg(['min','max']).diff(axis=1).reset_index()[['session','max']]
add_new_col(session_features, sessions_length, 'session_full_length', 'max', fillna=0, dtype='int')

sessions_starts = test_df.groupby(['session'])['ts'].agg('min').reset_index()
add_new_col(session_features, sessions_starts, 'session_first_ts', 'ts', fillna=-1, dtype='int')

sessions_ends = test_df.groupby(['session'])['ts'].agg('max').reset_index()
add_new_col(session_features, sessions_ends, 'session_last_ts', 'ts', fillna=-1, dtype='int')

del sessions_length, sessions_ends, sessions_starts
gc_clear()

In [9]:
test_df['d'] = test_df.groupby('session').ts.diff()
test_df.d = (test_df.d > 60*60*2).fillna(0).astype('int8')
test_df['d'] = test_df.groupby('session').d.cumsum()

real_sessions = (test_df.groupby(['session']).agg({'d':max})+1).reset_index()

add_new_col(session_features, real_sessions, 'session_real_num', 'd', fillna=0, dtype='int8')

del real_sessions
gc_clear()

In [10]:
real_sessions_length = test_df.groupby(['session','d']).agg({'ts':[min,max]}).diff(axis=1).reset_index()
real_sessions_length.columns = ['session','d','nan','sess_len']
real_sessions_length = real_sessions_length.groupby(['session']).agg({'sess_len':'mean'}).reset_index()

add_new_col(session_features, real_sessions_length, 'session_avg_real_length', 'sess_len', fillna=0)

del real_sessions_length
gc_clear()

In [11]:
items_per_real_session = test_df.groupby(['session','d']).agg({'aid':'nunique'}).reset_index()
items_per_real_session.columns = ['session','d','aid_unique']
items_per_real_session = items_per_real_session.groupby(['session']).agg({'aid_unique':'mean'}).reset_index()
items_per_real_session.columns = ['session','aid_avg']

add_new_col(session_features, items_per_real_session, 'session_avg_real_items_num', 'aid_avg', fillna=0)

del items_per_real_session
gc_clear()

In [12]:
for col in ['actions','clicks','carts','orders']:
    session_features[f'session_{col}_avg_real'] = session_features[f'session_{col}'] / session_features['session_real_num']

In [13]:
test_df['hour'] = (test_df['ts']%(24*3600))/3600
test_df['dow'] = (test_df['ts']%(7*24*3600))//(24*3600)

In [14]:
hour_by_action = test_df.groupby(['session','type']).agg({'hour':'mean'}).reset_index()
hour_by_action.columns = ['session','type','avg_hour']

for type_name, type_code in type_labels.items():
    add_new_col(
        session_features, 
        hour_by_action[hour_by_action['type']==type_code][['session','avg_hour']], 
        new_col = f'session_{type_name}_avg_hour', 
        group_col = 'avg_hour', 
        fillna = -1
    )
    
del hour_by_action
gc_clear()

In [15]:
test_df['tdiff'] = test_df.groupby(['session','d'])['ts'].transform('diff')
clicks_diff = test_df.groupby(['session']).agg({'tdiff':['mean','median']}).reset_index()
clicks_diff.columns = ['session','click_diff_mean','click_diff_median']

add_new_col(session_features, clicks_diff, 'session_click_diff_mean', 'click_diff_mean', fillna=-1)
add_new_col(session_features, clicks_diff, 'session_click_diff_median', 'click_diff_median', fillna=-1)

del clicks_diff, test_df['tdiff'] 
gc_clear()

In [16]:
first_df = test_df[['session','dow']].drop_duplicates(['session'], keep='first')
last_df = test_df[['session','dow']].drop_duplicates(['session'], keep='last')

add_new_col(session_features, first_df, 'session_start_dow', 'dow', fillna=-1, dtype='int8')
add_new_col(session_features, last_df, 'session_end_dow', 'dow', fillna=-1, dtype='int8')

del first_df, last_df 
gc_clear()

In [17]:
selected_feats = [
    'session',
    'session_actions',
    'session_avg_real_items_num',
    'session_avg_real_length',
    'session_carts',
    'session_carts_avg_hour',
    'session_carts_avg_real',
    'session_click_diff_mean',
    'session_clicks',
    'session_orders',
    'session_full_length',
    'session_items',
    'session_items_carted',
    'session_items_clicked',
    'session_last_ts'
]

In [18]:
session_features[selected_feats].to_parquet(f"feats/FE_sessions_{MODE}.pqt",index=False)

In [19]:
session_features[selected_feats]

,session,session_actions,session_avg_real_items_num,session_avg_real_length,session_carts,session_carts_avg_hour,session_carts_avg_real,session_click_diff_mean,session_clicks,session_orders,session_full_length,session_items,session_items_carted,session_items_clicked,session_last_ts
0,11098528,1,1.0,0.0,0,-1.00,0.0,-1.00000,1,0,0,1,0,1,1661119200
1,11098529,1,1.0,0.0,0,-1.00,0.0,-1.00000,1,0,0,1,0,1,1661119200
2,11098530,6,2.0,1332.0,1,22.37,1.0,266.40000,5,0,1332,2,1,2,1661120532
3,11098531,24,11.0,546.0,0,-1.00,0.0,23.73913,20,4,546,11,0,11,1661119746
4,11098532,2,2.0,795.0,0,-1.00,0.0,795.00000,2,0,795,2,0,2,1661119996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1801246,12899774,1,1.0,0.0,0,-1.00,0.0,-1.00000,1,0,0,1,0,1,1661723968
1801247,12899775,1,1.0,0.0,0,-1.00,0.0,-1.00000,1,0,0,1,0,1,1661723970
1801248,12899776,1,1.0,0.0,0,-1.00,0.0,-1.00000,1,0,0,1,0,1,1661723972
1801249,12899777,1,1.0,0.0,0,-1.00,0.0,-1.00000,1,0,0,1,0,1,1661723976


# SESSIONS TEST

In [20]:
MODE = 1

In [21]:
type_labels = {'clicks':0, 'carts':1, 'orders':2}

if MODE==1:
    dataset = "otto-chunk-data-inparquet-format"
if MODE==0:
    dataset = "otto-validation"

def load_test():    
    dfs = []
    for e, chunk_file in enumerate(glob.glob(f'input/{dataset}/test_parquet/*')):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True)

test_df = load_test()
# test_df = pd.read_parquet("input/otto-shimacos-validation/shimacos_test_0.parquet")
print('Test data has shape',test_df.shape)
print(f'{len(set(test_df.session))} unique sessions')

Test data has shape (6928123, 4)
1671803 unique sessions


In [22]:
test_df = test_df.sort_values(['session','ts'],ascending=[True,True]).reset_index(drop=True)
session_features = test_df[['session']].drop_duplicates().reset_index(drop=True)

In [23]:
def add_new_col(session_features, fromdf, new_col, group_col, fillna=0, dtype='float'):
    session_features[new_col] = session_features[
        'session'
    ].map(
        dict(zip(
            fromdf['session'],
            fromdf[group_col]
        ))
    ).fillna(fillna).astype(dtype)

In [24]:
items = test_df.groupby(['session']).agg({'aid':['count','nunique']}).reset_index()
items.columns = ['session','actions','items']

for new_col, group_col in zip(
    ['session_actions','session_items'],
    ['actions','items']
):
    add_new_col(session_features, items, new_col, group_col, fillna=0, dtype='int16')

del items
gc_clear()

In [25]:
items_by_action = test_df.groupby(['session','type']).agg({'aid':['count','nunique']}).reset_index()
items_by_action.columns = ['session','type','actions','items']

for type_name, type_code in type_labels.items():
    add_new_col(
        session_features, 
        items_by_action[items_by_action['type']==type_code][['session','actions']], 
        new_col = f'session_{type_name}', 
        group_col = 'actions', 
        fillna = 0, 
        dtype = 'int16'
    )
    add_new_col(
        session_features, 
        items_by_action[items_by_action['type']==type_code][['session','items']], 
        new_col = f'session_items_{type_name[:-1]}ed', 
        group_col = 'items', 
        fillna = 0, 
        dtype = 'int16'
    )
    
del items_by_action
gc_clear()

In [26]:
sessions_length = test_df.groupby(['session'])['ts'].agg(['min','max']).diff(axis=1).reset_index()[['session','max']]
add_new_col(session_features, sessions_length, 'session_full_length', 'max', fillna=0, dtype='int')

sessions_starts = test_df.groupby(['session'])['ts'].agg('min').reset_index()
add_new_col(session_features, sessions_starts, 'session_first_ts', 'ts', fillna=-1, dtype='int')

sessions_ends = test_df.groupby(['session'])['ts'].agg('max').reset_index()
add_new_col(session_features, sessions_ends, 'session_last_ts', 'ts', fillna=-1, dtype='int')

del sessions_length, sessions_ends, sessions_starts
gc_clear()

In [27]:
test_df['d'] = test_df.groupby('session').ts.diff()
test_df.d = (test_df.d > 60*60*2).fillna(0).astype('int8')
test_df['d'] = test_df.groupby('session').d.cumsum()

real_sessions = (test_df.groupby(['session']).agg({'d':max})+1).reset_index()

add_new_col(session_features, real_sessions, 'session_real_num', 'd', fillna=0, dtype='int8')

del real_sessions
gc_clear()

In [28]:
real_sessions_length = test_df.groupby(['session','d']).agg({'ts':[min,max]}).diff(axis=1).reset_index()
real_sessions_length.columns = ['session','d','nan','sess_len']
real_sessions_length = real_sessions_length.groupby(['session']).agg({'sess_len':'mean'}).reset_index()

add_new_col(session_features, real_sessions_length, 'session_avg_real_length', 'sess_len', fillna=0)

del real_sessions_length
gc_clear()

In [29]:
items_per_real_session = test_df.groupby(['session','d']).agg({'aid':'nunique'}).reset_index()
items_per_real_session.columns = ['session','d','aid_unique']
items_per_real_session = items_per_real_session.groupby(['session']).agg({'aid_unique':'mean'}).reset_index()
items_per_real_session.columns = ['session','aid_avg']

add_new_col(session_features, items_per_real_session, 'session_avg_real_items_num', 'aid_avg', fillna=0)

del items_per_real_session
gc_clear()

In [30]:
for col in ['actions','clicks','carts','orders']:
    session_features[f'session_{col}_avg_real'] = session_features[f'session_{col}'] / session_features['session_real_num']

In [31]:
test_df['hour'] = (test_df['ts']%(24*3600))/3600
test_df['dow'] = (test_df['ts']%(7*24*3600))//(24*3600)

In [32]:
hour_by_action = test_df.groupby(['session','type']).agg({'hour':'mean'}).reset_index()
hour_by_action.columns = ['session','type','avg_hour']

for type_name, type_code in type_labels.items():
    add_new_col(
        session_features, 
        hour_by_action[hour_by_action['type']==type_code][['session','avg_hour']], 
        new_col = f'session_{type_name}_avg_hour', 
        group_col = 'avg_hour', 
        fillna = -1
    )
    
del hour_by_action
gc_clear()

In [33]:
test_df['tdiff'] = test_df.groupby(['session','d'])['ts'].transform('diff')
clicks_diff = test_df.groupby(['session']).agg({'tdiff':['mean','median']}).reset_index()
clicks_diff.columns = ['session','click_diff_mean','click_diff_median']

add_new_col(session_features, clicks_diff, 'session_click_diff_mean', 'click_diff_mean', fillna=-1)
add_new_col(session_features, clicks_diff, 'session_click_diff_median', 'click_diff_median', fillna=-1)

del clicks_diff, test_df['tdiff'] 
gc_clear()

In [34]:
first_df = test_df[['session','dow']].drop_duplicates(['session'], keep='first')
last_df = test_df[['session','dow']].drop_duplicates(['session'], keep='last')

add_new_col(session_features, first_df, 'session_start_dow', 'dow', fillna=-1, dtype='int8')
add_new_col(session_features, last_df, 'session_end_dow', 'dow', fillna=-1, dtype='int8')

del first_df, last_df 
gc_clear()

In [35]:
selected_feats = [
    'session',
    'session_actions',
    'session_avg_real_items_num',
    'session_avg_real_length',
    'session_carts',
    'session_carts_avg_hour',
    'session_carts_avg_real',
    'session_click_diff_mean',
    'session_clicks',
    'session_orders',
    'session_full_length',
    'session_items',
    'session_items_carted',
    'session_items_clicked',
    'session_last_ts'
]

In [36]:
session_features[selected_feats].to_parquet(f"feats/FE_sessions_{MODE}.pqt",index=False)

In [37]:
session_features[selected_feats]

,session,session_actions,session_avg_real_items_num,session_avg_real_length,session_carts,session_carts_avg_hour,session_carts_avg_real,session_click_diff_mean,session_clicks,session_orders,session_full_length,session_items,session_items_carted,session_items_clicked,session_last_ts
0,12899779,1,1.00,0.0,0,-1.000000,0.00,-1.000000,1,0,0,1,0,1,1661724000
1,12899780,5,4.00,155.0,0,-1.000000,0.00,38.750000,5,0,155,4,0,4,1661724155
2,12899781,11,2.00,291.0,1,13.946944,0.25,166.285714,10,0,336160,5,1,5,1662060160
3,12899782,70,11.25,929.0,16,17.024531,4.00,56.303030,46,8,79953,38,14,32,1661803953
4,12899783,11,3.00,2178.0,0,-1.000000,0.00,816.750000,11,0,317140,9,0,9,1662041140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1671798,14571577,1,1.00,0.0,0,-1.000000,0.00,-1.000000,1,0,0,1,0,1,1662328774
1671799,14571578,1,1.00,0.0,0,-1.000000,0.00,-1.000000,1,0,0,1,0,1,1662328775
1671800,14571579,1,1.00,0.0,0,-1.000000,0.00,-1.000000,1,0,0,1,0,1,1662328775
1671801,14571580,1,1.00,0.0,0,-1.000000,0.00,-1.000000,1,0,0,1,0,1,1662328781


# ITEMS TRAIN

In [38]:
import pandas as pd, numpy as np
from tqdm.notebook import tqdm
import os, sys, pickle, glob, gc
from collections import Counter
import itertools
pd.set_option('display.max_columns', None)
from otto_utils import *

In [39]:
MODE = 0

In [40]:
if MODE==1:
    dataset = "otto-chunk-data-inparquet-format"
if MODE==0:
    dataset = "otto-validation"

type_labels = {'clicks':0, 'carts':1, 'orders':2}

def load_df(name='test'):    
    dfs = []
    for e, chunk_file in enumerate(glob.glob(f'input/{dataset}/{name}_parquet/*')):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True)

test_df = load_df("test")
print('Test data has shape',test_df.shape)
print(f'{len(set(test_df.session))} unique sessions')

train_df = load_df("train")
print('Train data has shape',train_df.shape)
print(f'{len(set(train_df.session))} unique sessions')

Test data has shape (7683577, 4)
1801251 unique sessions
Train data has shape (163955180, 4)
11098528 unique sessions


In [41]:
def get_fe_1(df, sfx, unique=True):
    dct = {0:'CL', 1:'CA', 2:'OR'}
    
    if unique:
        df.drop_duplicates(subset=['session','aid','type'], inplace=True)
    
    t = df.groupby(['aid','type']).agg({'session':'count'}).reset_index()
    t = pd.pivot_table(
        t, 
        values='session', 
        index=['aid'], 
        columns=['type'], 
        aggfunc=np.sum, 
        fill_value=0
    ).reset_index()
    t.columns = ['aid','num_0','num_1','num_2']
    
    t['aid_CL2CA'] = t['num_1']/t['num_0']
    t['aid_CA2OR'] = t['num_2']/t['num_1']
    t['aid_CL2OR'] = t['num_2']/t['num_0']

    for i in [0,1,2]:
        t[f'aid_{dct[i]}_vs_mean'] = t[f'num_{i}'] / t[f'num_{i}'].mean()
        for pct in [False,True]:
            t[
                f'aid_{dct[i]}_rank_{"pct" if pct else "int"}'
            ] = t[f'num_{i}'].rank(method='average', ascending=False, pct=pct)
        del t[f'num_{i}']
    
    t.columns = ["aid"] + [c+"_"+sfx for c in t.columns if c!='aid']
    
    return t

In [42]:
tst = get_fe_1(test_df.copy(), sfx='tst', unique=True)
trn = get_fe_1(train_df.copy(), sfx='trn', unique=True)
feats_df = trn.merge(tst,on=['aid'],how='outer')

for c in tst.columns:
    if c!='aid':
        feats_df[c[:-3]+"tst_vs_trn"] = feats_df[c[:-3]+"tst"] / feats_df[c[:-3]+"trn"]
        
del trn, tst
gc_clear()


for sfx in ['trn','tst','tst_vs_trn']:
    for act in ['CL2CA','CA2OR','CL2OR']:
        feats_df[f'aid_{act}_{sfx}'].fillna(-1,inplace=True)
    for act in ['CL','CA','OR']:
        feats_df[f'aid_{act}_vs_mean_{sfx}'].fillna(-1,inplace=True)
        feats_df[f'aid_{act}_rank_int_{sfx}'].fillna(2e6,inplace=True)
        feats_df[f'aid_{act}_rank_pct_{sfx}'].fillna(2,inplace=True)
        
feats_df

,aid,aid_CL2CA_trn,aid_CA2OR_trn,aid_CL2OR_trn,aid_CL_vs_mean_trn,aid_CL_rank_int_trn,aid_CL_rank_pct_trn,aid_CA_vs_mean_trn,aid_CA_rank_int_trn,aid_CA_rank_pct_trn,aid_OR_vs_mean_trn,aid_OR_rank_int_trn,aid_OR_rank_pct_trn,aid_CL2CA_tst,aid_CA2OR_tst,aid_CL2OR_tst,aid_CL_vs_mean_tst,aid_CL_rank_int_tst,aid_CL_rank_pct_tst,aid_CA_vs_mean_tst,aid_CA_rank_int_tst,aid_CA_rank_pct_tst,aid_OR_vs_mean_tst,aid_OR_rank_int_tst,aid_OR_rank_pct_tst,aid_CL2CA_tst_vs_trn,aid_CA2OR_tst_vs_trn,aid_CL2OR_tst_vs_trn,aid_CL_vs_mean_tst_vs_trn,aid_CL_rank_int_tst_vs_trn,aid_CL_rank_pct_tst_vs_trn,aid_CA_vs_mean_tst_vs_trn,aid_CA_rank_int_tst_vs_trn,aid_CA_rank_pct_tst_vs_trn,aid_OR_vs_mean_tst_vs_trn,aid_OR_rank_int_tst_vs_trn,aid_OR_rank_pct_tst_vs_trn
0,0,0.000000,-1.000000,0.000000,0.471271,475077.0,0.260245,0.000000,1464080.0,0.802016,0.000000,1198109.0,0.656319,0.000000,-1.000000,0.00000,0.643156,216321.0,0.247266,0.000000,553715.5,0.632925,0.000000,461001.0,0.526947,-1.000000,-1.000000,-1.000000,1.364727,4.553388e-01,0.950127,-1.000000,3.782003e-01,0.789167,-1.00000,3.847738e-01,0.802884
1,1,0.035714,0.000000,0.000000,0.507523,449986.5,0.246501,0.169307,905471.0,0.496013,0.000000,1198109.0,0.656319,-1.000000,-1.000000,-1.00000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,-1.000000,-1.000000,-1.000000,2.000000e+06,2.000000,-1.000000,2.000000e+06,2.000000,-1.00000,2.000000e+06,2.000000
2,2,0.000000,-1.000000,0.000000,0.217510,811572.0,0.444575,0.000000,1464080.0,0.802016,0.000000,1198109.0,0.656319,0.000000,-1.000000,0.00000,0.482367,280397.0,0.320508,0.000000,553715.5,0.632925,0.000000,461001.0,0.526947,-1.000000,-1.000000,-1.000000,2.217681,3.454986e-01,0.720930,-1.000000,3.782003e-01,0.789167,-1.00000,3.847738e-01,0.802884
3,3,0.091317,0.229508,0.020958,12.108038,22526.5,0.012340,10.327711,28278.0,0.015491,7.163523,45613.0,0.024987,0.103261,0.105263,0.01087,29.585182,2506.0,0.002864,29.096598,2558.5,0.002924,24.786151,6960.5,0.007956,1.130791,0.458647,0.518634,2.443433,1.112468e-01,0.232132,2.817333,9.047670e-02,0.188792,3.46005,1.525990e-01,0.318419
4,4,0.051546,0.000000,0.000000,1.758203,167525.5,0.091770,0.846534,313174.0,0.171555,0.000000,1198109.0,0.656319,0.000000,-1.000000,0.00000,0.964734,149199.5,0.170543,0.000000,553715.5,0.632925,0.000000,461001.0,0.526947,0.000000,-1.000000,-1.000000,0.548705,8.906077e-01,1.858375,0.000000,1.768076e+00,3.689334,-1.00000,3.847738e-01,0.802884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1844279,1855264,-1.000000,-1.000000,-1.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,0.000000,-1.000000,0.00000,0.643156,216321.0,0.247266,0.000000,553715.5,0.632925,0.000000,461001.0,0.526947,-1.000000,-1.000000,-1.000000,-1.000000,2.000000e+06,2.000000,-1.000000,2.000000e+06,2.000000,-1.00000,2.000000e+06,2.000000
1844280,1855363,-1.000000,-1.000000,-1.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,0.000000,-1.000000,0.00000,0.160789,672087.0,0.768229,0.000000,553715.5,0.632925,0.000000,461001.0,0.526947,-1.000000,-1.000000,-1.000000,-1.000000,2.000000e+06,2.000000,-1.000000,2.000000e+06,2.000000,-1.00000,2.000000e+06,2.000000
1844281,1855496,-1.000000,-1.000000,-1.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,0.000000,-1.000000,0.00000,0.803945,176581.0,0.201841,0.000000,553715.5,0.632925,0.000000,461001.0,0.526947,-1.000000,-1.000000,-1.000000,-1.000000,2.000000e+06,2.000000,-1.000000,2.000000e+06,2.000000,-1.00000,2.000000e+06,2.000000
1844282,1855514,-1.000000,-1.000000,-1.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,1.000000,0.000000,0.00000,0.160789,672087.0,0.768229,1.531400,157947.0,0.180541,0.000000,461001.0,0.526947,-1.000000,-1.000000,-1.000000,-1.000000

In [43]:
def get_ratings(df,sfx):
    for type_name, type_code in type_labels.items():
        tops = pd.DataFrame(
            df.loc[df['type']==type_code,'aid'].value_counts()
        ).reset_index()
        tops.columns = ['aid',type_name]
        tops.reset_index(inplace=True)
        tops.columns = [f'{type_name}_rating_{sfx}','aid',type_name]
        
        if type_code==0:
            out = tops[['aid',f'{type_name}_rating_{sfx}']]
        else:
            out = out.merge(
                tops[['aid',f'{type_name}_rating_{sfx}']],
                on = ['aid'], how='outer'
            )
    del tops
    gc_clear()
    return out

In [44]:
ratings = get_ratings(test_df,"test")
ratings = ratings.merge(
    get_ratings(train_df,"train"),
    on=['aid'],
    how='outer'
)
ratings = ratings.merge(
    get_ratings(pd.concat([train_df,test_df]),"full"),
    on=['aid'],
    how='outer'
)


feats_df = feats_df.merge(ratings, on=['aid'], how='outer')

del ratings
gc_clear()

for sfx in ['test','train','full']:
    for act in ['clicks','carts','orders']:
        feats_df[f"{act}_rating_{sfx}"] = feats_df[f"{act}_rating_{sfx}"].fillna(2000000).astype(int)
        
feats_df

,aid,aid_CL2CA_trn,aid_CA2OR_trn,aid_CL2OR_trn,aid_CL_vs_mean_trn,aid_CL_rank_int_trn,aid_CL_rank_pct_trn,aid_CA_vs_mean_trn,aid_CA_rank_int_trn,aid_CA_rank_pct_trn,aid_OR_vs_mean_trn,aid_OR_rank_int_trn,aid_OR_rank_pct_trn,aid_CL2CA_tst,aid_CA2OR_tst,aid_CL2OR_tst,aid_CL_vs_mean_tst,aid_CL_rank_int_tst,aid_CL_rank_pct_tst,aid_CA_vs_mean_tst,aid_CA_rank_int_tst,aid_CA_rank_pct_tst,aid_OR_vs_mean_tst,aid_OR_rank_int_tst,aid_OR_rank_pct_tst,aid_CL2CA_tst_vs_trn,aid_CA2OR_tst_vs_trn,aid_CL2OR_tst_vs_trn,aid_CL_vs_mean_tst_vs_trn,aid_CL_rank_int_tst_vs_trn,aid_CL_rank_pct_tst_vs_trn,aid_CA_vs_mean_tst_vs_trn,aid_CA_rank_int_tst_vs_trn,aid_CA_rank_pct_tst_vs_trn,aid_OR_vs_mean_tst_vs_trn,aid_OR_rank_int_tst_vs_trn,aid_OR_rank_pct_tst_vs_trn,clicks_rating_test,carts_rating_test,orders_rating_test,clicks_rating_train,carts_rating_train,orders_rating_train,clicks_rating_full,carts_rating_full,orders_rating_full
0,0,0.000000,-1.000000,0.000000,0.471271,475077.0,0.260245,0.000000,1464080.0,0.802016,0.000000,1198109.0,0.656319,0.000000,-1.000000,0.00000,0.643156,216321.0,0.247266,0.000000,553715.5,0.632925,0.000000,461001.0,0.526947,-1.000000,-1.000000,-1.000000,1.364727,4.553388e-01,0.950127,-1.000000,3.782003e-01,0.789167,-1.00000,3.847738e-01,0.802884,225457,2000000,2000000,508675,2000000,2000000,471766,2000000,2000000
1,1,0.035714,0.000000,0.000000,0.507523,449986.5,0.246501,0.169307,905471.0,0.496013,0.000000,1198109.0,0.656319,-1.000000,-1.000000,-1.00000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,-1.000000,-1.000000,-1.000000,2.000000e+06,2.000000,-1.000000,2.000000e+06,2.000000,-1.00000,2.000000e+06,2.000000,2000000,2000000,2000000,529680,945282,2000000,552240,1107901,2000000
2,2,0.000000,-1.000000,0.000000,0.217510,811572.0,0.444575,0.000000,1464080.0,0.802016,0.000000,1198109.0,0.656319,0.000000,-1.000000,0.00000,0.482367,280397.0,0.320508,0.000000,553715.5,0.632925,0.000000,461001.0,0.526947,-1.000000,-1.000000,-1.000000,2.217681,3.454986e-01,0.720930,-1.000000,3.782003e-01,0.789167,-1.00000,3.847738e-01,0.802884,279939,2000000,2000000,963819,2000000,2000000,858280,2000000,2000000
3,3,0.091317,0.229508,0.020958,12.108038,22526.5,0.012340,10.327711,28278.0,0.015491,7.163523,45613.0,0.024987,0.103261,0.105263,0.01087,29.585182,2506.0,0.002864,29.096598,2558.5,0.002924,24.786151,6960.5,0.007956,1.130791,0.458647,0.518634,2.443433,1.112468e-01,0.232132,2.817333,9.047670e-02,0.188792,3.46005,1.525990e-01,0.318419,1808,1873,8333,19306,31460,46642,15145,21770,40705
4,4,0.051546,0.000000,0.000000,1.758203,167525.5,0.091770,0.846534,313174.0,0.171555,0.000000,1198109.0,0.656319,0.000000,-1.000000,0.00000,0.964734,149199.5,0.170543,0.000000,553715.5,0.632925,0.000000,461001.0,0.526947,0.000000,-1.000000,-1.000000,0.548705,8.906077e-01,1.858375,0.000000,1.768076e+00,3.689334,-1.00000,3.847738e-01,0.802884,138716,2000000,2000000,171060,339938,2000000,169142,378962,2000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1844279,1855264,-1.000000,-1.000000,-1.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,0.000000,-1.000000,0.00000,0.643156,216321.0,0.247266,0.000000,553715.5,0.632925,0.000000,461001.0,0.526947,-1.000000,-1.000000,-1.000000,-1.000000,2.000000e+06,2.000000,-1.000000,2.000000e+06,2.000000,-1.00000,2.000000e+06,2.000000,243909,2000000,2000000,2000000,2000000,2000000,1728961,2000000,2000000
1844280,1855363,-1.000000,-1.000000,-1.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,0.000000,-1.000000,0.00000,0.160789,672087.0,0.768229,0.000000,553715.5,0.632925,0.000000,461001.0,0.526947,-1.000000,-1.000000,-1.000000,-1.000000,2.000000e+06,2.000000,-1.000000,2.000000e+06,2.000000,-1.00000,2.000000e+06,2.000000,466625,2000000,2000000,2000000,2000000,200

In [45]:
def get_multi_percent(df,sfx):
    for type_name, type_code in type_labels.items():
        t = df[df['type']==type_code].groupby(['aid','session'],as_index=False).agg({'ts':'count'})
        t['ts'] = (t['ts']>1).astype('int8')
        t = t.groupby(['aid'],as_index=False).agg({'ts':['count','sum']})
        t.columns = ['aid','users','multi_users']
        t[f'aid_multi_{type_name}_percent_{sfx}'] = t['multi_users']/t['users']
        del t['users'], t['multi_users']
        if type_code==0:
            out = t
        else:
            out = out.merge(t,on=['aid'],how='outer')
    del t
    gc_clear()
    return out

In [46]:
multi = get_multi_percent(test_df,"test")
multi = multi.merge(
    get_multi_percent(train_df,"train"),
    on=['aid'],
    how='outer'
)
multi = multi.merge(
    get_multi_percent(pd.concat([train_df,test_df]),"full"),
    on=['aid'],
    how='outer'
)


feats_df = feats_df.merge(multi, on=['aid'], how='outer')

del multi
gc_clear()

for sfx in ['test','train','full']:
    for act in ['clicks','carts','orders']:
        feats_df[f"aid_multi_{act}_percent_{sfx}"] = feats_df[f"aid_multi_{act}_percent_{sfx}"].fillna(-1)
        
        
feats_df

,aid,aid_CL2CA_trn,aid_CA2OR_trn,aid_CL2OR_trn,aid_CL_vs_mean_trn,aid_CL_rank_int_trn,aid_CL_rank_pct_trn,aid_CA_vs_mean_trn,aid_CA_rank_int_trn,aid_CA_rank_pct_trn,aid_OR_vs_mean_trn,aid_OR_rank_int_trn,aid_OR_rank_pct_trn,aid_CL2CA_tst,aid_CA2OR_tst,aid_CL2OR_tst,aid_CL_vs_mean_tst,aid_CL_rank_int_tst,aid_CL_rank_pct_tst,aid_CA_vs_mean_tst,aid_CA_rank_int_tst,aid_CA_rank_pct_tst,aid_OR_vs_mean_tst,aid_OR_rank_int_tst,aid_OR_rank_pct_tst,aid_CL2CA_tst_vs_trn,aid_CA2OR_tst_vs_trn,aid_CL2OR_tst_vs_trn,aid_CL_vs_mean_tst_vs_trn,aid_CL_rank_int_tst_vs_trn,aid_CL_rank_pct_tst_vs_trn,aid_CA_vs_mean_tst_vs_trn,aid_CA_rank_int_tst_vs_trn,aid_CA_rank_pct_tst_vs_trn,aid_OR_vs_mean_tst_vs_trn,aid_OR_rank_int_tst_vs_trn,aid_OR_rank_pct_tst_vs_trn,clicks_rating_test,carts_rating_test,orders_rating_test,clicks_rating_train,carts_rating_train,orders_rating_train,clicks_rating_full,carts_rating_full,orders_rating_full,aid_multi_clicks_percent_test,aid_multi_carts_percent_test,aid_multi_orders_percent_test,aid_multi_clicks_percent_train,aid_multi_carts_percent_train,aid_multi_orders_percent_train,aid_multi_clicks_percent_full,aid_multi_carts_percent_full,aid_multi_orders_percent_full
0,0,0.000000,-1.000000,0.000000,0.471271,475077.0,0.260245,0.000000,1464080.0,0.802016,0.000000,1198109.0,0.656319,0.000000,-1.000000,0.00000,0.643156,216321.0,0.247266,0.000000,553715.5,0.632925,0.000000,461001.0,0.526947,-1.000000,-1.000000,-1.000000,1.364727,4.553388e-01,0.950127,-1.000000,3.782003e-01,0.789167,-1.00000,3.847738e-01,0.802884,225457,2000000,2000000,508675,2000000,2000000,471766,2000000,2000000,0.250000,-1.000000,-1.0,0.230769,-1.000000,-1.000000,0.233333,-1.000,-1.0000
1,1,0.035714,0.000000,0.000000,0.507523,449986.5,0.246501,0.169307,905471.0,0.496013,0.000000,1198109.0,0.656319,-1.000000,-1.000000,-1.00000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,-1.000000,-1.000000,-1.000000,2.000000e+06,2.000000,-1.000000,2.000000e+06,2.000000,-1.00000,2.000000e+06,2.000000,2000000,2000000,2000000,529680,945282,2000000,552240,1107901,2000000,-1.000000,-1.000000,-1.0,0.107143,0.000000,-1.000000,0.107143,0.000,-1.0000
2,2,0.000000,-1.000000,0.000000,0.217510,811572.0,0.444575,0.000000,1464080.0,0.802016,0.000000,1198109.0,0.656319,0.000000,-1.000000,0.00000,0.482367,280397.0,0.320508,0.000000,553715.5,0.632925,0.000000,461001.0,0.526947,-1.000000,-1.000000,-1.000000,2.217681,3.454986e-01,0.720930,-1.000000,3.782003e-01,0.789167,-1.00000,3.847738e-01,0.802884,279939,2000000,2000000,963819,2000000,2000000,858280,2000000,2000000,0.333333,-1.000000,-1.0,0.000000,-1.000000,-1.000000,0.066667,-1.000,-1.0000
3,3,0.091317,0.229508,0.020958,12.108038,22526.5,0.012340,10.327711,28278.0,0.015491,7.163523,45613.0,0.024987,0.103261,0.105263,0.01087,29.585182,2506.0,0.002864,29.096598,2558.5,0.002924,24.786151,6960.5,0.007956,1.130791,0.458647,0.518634,2.443433,1.112468e-01,0.232132,2.817333,9.047670e-02,0.188792,3.46005,1.525990e-01,0.318419,1808,1873,8333,19306,31460,46642,15145,21770,40705,0.326087,0.210526,0.0,0.359281,0.032787,0.071429,0.352113,0.075,0.0625
4,4,0.051546,0.000000,0.000000,1.758203,167525.5,0.091770,0.846534,313174.0,0.171555,0.000000,1198109.0,0.656319,0.000000,-1.000000,0.00000,0.964734,149199.5,0.170543,0.000000,553715.5,0.632925,0.000000,461001.0,0.526947,0.000000,-1.000000,-1.000000,0.548705,8.906077e-01,1.858375,0.000000,1.768076e+00,3.689334,-1.00000,3.847738e-01,0.802884,138716,2000000,2000000,171060,339938,2000000,169142,378962,2000000,0.166667,-1.000000,-1.0,0.216495,0.000000,-1.000000,0.213592,0.000,-1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1844279,1855264,-1.000000,-1.000000,-1.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,0.000000,-1.000000,0.00000,0.6

In [47]:
def get_favourite_dows(df,sfx):
    df['dow'] = ((df['ts']%(7*24*60*60))//(24*60*60)).astype('int8')
    for type_name, type_code in type_labels.items():
        fav = df[
            df['type']==type_code
        ].groupby(
            ['aid','dow'],
            as_index=False
        ).agg(
            {'session': 'count'}
        ).sort_values(
            by=['aid','session'],
            ascending=[True,False]
        ).drop_duplicates(['aid'])
        del fav['session']
        fav.columns = ['aid',f'aid_{type_name}_favourite_dow_{sfx}']
        if type_code==0:
            out = fav
        else:
            out = out.merge(fav,on=['aid'],how='outer')
    del df['dow'], fav
    gc_clear()
    return out

In [48]:
fav = get_favourite_dows(test_df,"test")
fav = fav.merge(
    get_favourite_dows(train_df,"train"),
    on=['aid'],
    how='outer'
)
fav = fav.merge(
    get_favourite_dows(pd.concat([train_df,test_df]),"full"),
    on=['aid'],
    how='outer'
)

feats_df = feats_df.merge(fav, on=['aid'], how='outer')

del fav
gc_clear()

In [49]:
selected_feats = [
    'aid',
    'aid_CA2OR_trn',
    'aid_CA_rank_int_tst_vs_trn',
    'aid_CA_vs_mean_trn',
    'aid_CA_vs_mean_tst',
    'aid_CA_vs_mean_tst_vs_trn',
    'aid_CL2CA_trn',
    'aid_CL2CA_tst',
    'aid_CL2OR_trn',
    'aid_CL_rank_int_trn',
    'aid_CL_rank_int_tst_vs_trn',
    'aid_CL_rank_pct_tst_vs_trn',
    'aid_CL_vs_mean_trn',
    'aid_CL_vs_mean_tst_vs_trn',
    'aid_multi_clicks_percent_full',
    'aid_multi_orders_percent_train',
    'aid_clicks_favourite_dow_test',
    'carts_rating_full',
    'carts_rating_train',
    'clicks_rating_full',
    'clicks_rating_train',
    'orders_rating_full'
]

In [50]:
feats_df[selected_feats].to_parquet(f'feats/FE_aids_{MODE}.pqt',index=False)

In [51]:
feats_df[selected_feats]

,aid,aid_CA2OR_trn,aid_CA_rank_int_tst_vs_trn,aid_CA_vs_mean_trn,aid_CA_vs_mean_tst,aid_CA_vs_mean_tst_vs_trn,aid_CL2CA_trn,aid_CL2CA_tst,aid_CL2OR_trn,aid_CL_rank_int_trn,aid_CL_rank_int_tst_vs_trn,aid_CL_rank_pct_tst_vs_trn,aid_CL_vs_mean_trn,aid_CL_vs_mean_tst_vs_trn,aid_multi_clicks_percent_full,aid_multi_orders_percent_train,aid_clicks_favourite_dow_test,carts_rating_full,carts_rating_train,clicks_rating_full,clicks_rating_train,orders_rating_full
0,0,-1.000000,3.782003e-01,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,475077.0,4.553388e-01,0.950127,0.471271,1.364727,0.233333,-1.000000,6.0,2000000,2000000,471766,508675,2000000
1,1,0.000000,2.000000e+06,0.169307,-1.000000,-1.000000,0.035714,-1.000000,0.000000,449986.5,2.000000e+06,2.000000,0.507523,-1.000000,0.107143,-1.000000,NaN,1107901,945282,552240,529680,2000000
2,2,-1.000000,3.782003e-01,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,811572.0,3.454986e-01,0.720930,0.217510,2.217681,0.066667,-1.000000,4.0,2000000,2000000,858280,963819,2000000
3,3,0.229508,9.047670e-02,10.327711,29.096598,2.817333,0.091317,0.103261,0.020958,22526.5,1.112468e-01,0.232132,12.108038,2.443433,0.352113,0.071429,3.0,21770,31460,15145,19306,40705
4,4,0.000000,1.768076e+00,0.846534,0.000000,0.000000,0.051546,0.000000,0.000000,167525.5,8.906077e-01,1.858375,1.758203,0.548705,0.213592,-1.000000,3.0,378962,339938,169142,171060,2000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1844279,1855264,-1.000000,2.000000e+06,-1.000000,0.000000,-1.000000,-1.000000,0.000000,-1.000000,2000000.0,2.000000e+06,2.000000,-1.000000,-1.000000,0.000000,-1.000000,3.0,2000000,2000000,1728961,2000000,2000000
1844280,1855363,-1.000000,2.000000e+06,-1.000000,0.000000,-1.000000,-1.000000,0.000000,-1.000000,2000000.0,2.000000e+06,2.000000,-1.000000,-1.000000,1.000000,-1.000000,3.0,2000000,2000000,1802240,2000000,2000000
1844281,1855496,-1.000000,2.000000e+06,-1.000000,0.000000,-1.000000,-1.000000,0.000000,-1.000000,2000000.0,2.000000e+06,2.000000,-1.000000,-1.000000,0.000000,-1.000000,0.0,2000000,2000000,1650368,2000000,2000000
1844282,1855514,-1.000000,2.000000e+06,-1.000000,1.531400,-1.000000,-1.000000,1.000000,-1.000000,2000000.0,2.000000e+06,2.000000,-1.000000,-1.000000,1.000000,-1.000000,2.0,817033,2000000,1749223,2000000,2000000


# ITEMS TEST

In [52]:
MODE = 1

In [53]:
if MODE==1:
    dataset = "otto-chunk-data-inparquet-format"
if MODE==0:
    dataset = "otto-validation"

type_labels = {'clicks':0, 'carts':1, 'orders':2}

def load_df(name='test'):    
    dfs = []
    for e, chunk_file in enumerate(glob.glob(f'input/{dataset}/{name}_parquet/*')):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True)

test_df = load_df("test")
print('Test data has shape',test_df.shape)
print(f'{len(set(test_df.session))} unique sessions')

train_df = load_df("train")
print('Train data has shape',train_df.shape)
print(f'{len(set(train_df.session))} unique sessions')

Test data has shape (6928123, 4)
1671803 unique sessions
Train data has shape (216716096, 4)
12899779 unique sessions


In [54]:
def get_fe_1(df, sfx, unique=True):
    dct = {0:'CL', 1:'CA', 2:'OR'}
    
    if unique:
        df.drop_duplicates(subset=['session','aid','type'], inplace=True)
    
    t = df.groupby(['aid','type']).agg({'session':'count'}).reset_index()
    t = pd.pivot_table(
        t, 
        values='session', 
        index=['aid'], 
        columns=['type'], 
        aggfunc=np.sum, 
        fill_value=0
    ).reset_index()
    t.columns = ['aid','num_0','num_1','num_2']
    
    t['aid_CL2CA'] = t['num_1']/t['num_0']
    t['aid_CA2OR'] = t['num_2']/t['num_1']
    t['aid_CL2OR'] = t['num_2']/t['num_0']

    for i in [0,1,2]:
        t[f'aid_{dct[i]}_vs_mean'] = t[f'num_{i}'] / t[f'num_{i}'].mean()
        for pct in [False,True]:
            t[
                f'aid_{dct[i]}_rank_{"pct" if pct else "int"}'
            ] = t[f'num_{i}'].rank(method='average', ascending=False, pct=pct)
        del t[f'num_{i}']
    
    t.columns = ["aid"] + [c+"_"+sfx for c in t.columns if c!='aid']
    
    return t

In [55]:
tst = get_fe_1(test_df.copy(), sfx='tst', unique=True)
trn = get_fe_1(train_df.copy(), sfx='trn', unique=True)
feats_df = trn.merge(tst,on=['aid'],how='outer')

for c in tst.columns:
    if c!='aid':
        feats_df[c[:-3]+"tst_vs_trn"] = feats_df[c[:-3]+"tst"] / feats_df[c[:-3]+"trn"]
        
del trn, tst
gc_clear()


for sfx in ['trn','tst','tst_vs_trn']:
    for act in ['CL2CA','CA2OR','CL2OR']:
        feats_df[f'aid_{act}_{sfx}'].fillna(-1,inplace=True)
    for act in ['CL','CA','OR']:
        feats_df[f'aid_{act}_vs_mean_{sfx}'].fillna(-1,inplace=True)
        feats_df[f'aid_{act}_rank_int_{sfx}'].fillna(2e6,inplace=True)
        feats_df[f'aid_{act}_rank_pct_{sfx}'].fillna(2,inplace=True)
        
feats_df

,aid,aid_CL2CA_trn,aid_CA2OR_trn,aid_CL2OR_trn,aid_CL_vs_mean_trn,aid_CL_rank_int_trn,aid_CL_rank_pct_trn,aid_CA_vs_mean_trn,aid_CA_rank_int_trn,aid_CA_rank_pct_trn,aid_OR_vs_mean_trn,aid_OR_rank_int_trn,aid_OR_rank_pct_trn,aid_CL2CA_tst,aid_CA2OR_tst,aid_CL2OR_tst,aid_CL_vs_mean_tst,aid_CL_rank_int_tst,aid_CL_rank_pct_tst,aid_CA_vs_mean_tst,aid_CA_rank_int_tst,aid_CA_rank_pct_tst,aid_OR_vs_mean_tst,aid_OR_rank_int_tst,aid_OR_rank_pct_tst,aid_CL2CA_tst_vs_trn,aid_CA2OR_tst_vs_trn,aid_CL2OR_tst_vs_trn,aid_CL_vs_mean_tst_vs_trn,aid_CL_rank_int_tst_vs_trn,aid_CL_rank_pct_tst_vs_trn,aid_CA_vs_mean_tst_vs_trn,aid_CA_rank_int_tst_vs_trn,aid_CA_rank_pct_tst_vs_trn,aid_OR_vs_mean_tst_vs_trn,aid_OR_rank_int_tst_vs_trn,aid_OR_rank_pct_tst_vs_trn
0,0,0.000000,-1.000000,0.000000,0.506732,460602.0,0.248222,0.000000,1545169.5,0.832705,0.000000,1256772.0,0.677285,0.000000,-1.000000,0.000000,0.636198,199543.5,0.254687,0.000000,496451.0,0.633644,0.000000,413098.0,0.527256,-1.000000,-1.000000,-1.000000,1.255491,4.332233e-01,1.026043,-1.000000,3.212923e-01,0.760946,-1.000000,3.286976e-01,0.778485
1,1,0.034483,0.000000,0.000000,0.408201,539951.5,0.290984,0.130085,1035279.0,0.557921,0.000000,1256772.0,0.677285,-1.000000,-1.000000,-1.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,-1.000000,-1.000000,-1.000000,2.000000e+06,2.000000,-1.000000,2.000000e+06,2.000000,-1.000000,2.000000e+06,2.000000
2,2,0.000000,-1.000000,0.000000,0.225214,813738.5,0.438530,0.000000,1545169.5,0.832705,0.000000,1256772.0,0.677285,-1.000000,-1.000000,-1.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,-1.000000,-1.000000,-1.000000,2.000000e+06,2.000000,-1.000000,2.000000e+06,2.000000,-1.000000,2.000000e+06,2.000000
3,3,0.122161,0.217949,0.026625,17.974922,13652.0,0.007357,20.293184,11903.0,0.006415,13.444350,23275.5,0.012543,0.147826,0.117647,0.017391,18.290694,4520.5,0.005770,26.793551,2626.0,0.003352,25.352252,6047.5,0.007719,1.210089,0.539792,0.653197,1.017567,3.311236e-01,0.784231,1.320323,2.206167e-01,0.522507,1.885718,2.598226e-01,0.615362
4,4,0.065217,0.000000,0.000000,1.942474,156592.0,0.084389,1.170761,246221.0,0.132691,0.000000,1256772.0,0.677285,0.000000,-1.000000,0.000000,0.795248,162864.5,0.207872,0.000000,496451.0,0.633644,0.000000,413098.0,0.527256,0.000000,-1.000000,-1.000000,0.409399,1.040056e+00,2.463262,0.000000,2.016282e+00,4.775349,-1.000000,3.286976e-01,0.778485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855598,1855598,0.000000,-1.000000,0.000000,0.098531,1334178.5,0.719000,0.000000,1545169.5,0.832705,0.000000,1256772.0,0.677285,-1.000000,-1.000000,-1.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,-1.000000,-1.000000,-1.000000,2.000000e+06,2.000000,-1.000000,2.000000e+06,2.000000,-1.000000,2.000000e+06,2.000000
1855599,1855599,0.000000,-1.000000,0.000000,0.154835,1030696.0,0.555451,0.000000,1545169.5,0.832705,0.000000,1256772.0,0.677285,0.000000,-1.000000,0.000000,0.159050,605034.0,0.772233,0.000000,496451.0,0.633644,0.000000,413098.0,0.527256,-1.000000,-1.000000,-1.000000,1.027220,5.870150e-01,1.390282,-1.000000,3.212923e-01,0.760946,-1.000000,3.286976e-01,0.778485
1855600,1855600,0.107143,0.166667,0.017857,0.788250,329754.5,0.177707,0.780507,337408.5,0.181832,0.395422,510666.5,0.275202,1.000000,0.000000,0.000000,0.159050,605034.0,0.772233,1.576091,142235.0,0.181541,0.000000,413098.0,0.527256,9.333333,0.000000,0.000000,0.201775,1.834801e+00,4.345531,2.019317,4.215513e-01,0.998399,0.000000,8.089389e-01,1.915885
1855601,1855601,0.116667,0.000000,0.000000,0.844554,312364.0,0.168336,0.910592,299774.5,0.161551,0.000000,1256772.0,0.677285,-1.000000,-1.000000,-1.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,-1.000000,-1.000000

In [56]:
def get_ratings(df,sfx):
    for type_name, type_code in type_labels.items():
        tops = pd.DataFrame(
            df.loc[df['type']==type_code,'aid'].value_counts()
        ).reset_index()
        tops.columns = ['aid',type_name]
        tops.reset_index(inplace=True)
        tops.columns = [f'{type_name}_rating_{sfx}','aid',type_name]
        
        if type_code==0:
            out = tops[['aid',f'{type_name}_rating_{sfx}']]
        else:
            out = out.merge(
                tops[['aid',f'{type_name}_rating_{sfx}']],
                on = ['aid'], how='outer'
            )
    del tops
    gc_clear()
    return out

In [57]:
ratings = get_ratings(test_df,"test")
ratings = ratings.merge(
    get_ratings(train_df,"train"),
    on=['aid'],
    how='outer'
)
ratings = ratings.merge(
    get_ratings(pd.concat([train_df,test_df]),"full"),
    on=['aid'],
    how='outer'
)


feats_df = feats_df.merge(ratings, on=['aid'], how='outer')

del ratings
gc_clear()

for sfx in ['test','train','full']:
    for act in ['clicks','carts','orders']:
        feats_df[f"{act}_rating_{sfx}"] = feats_df[f"{act}_rating_{sfx}"].fillna(2000000).astype(int)
        
feats_df

,aid,aid_CL2CA_trn,aid_CA2OR_trn,aid_CL2OR_trn,aid_CL_vs_mean_trn,aid_CL_rank_int_trn,aid_CL_rank_pct_trn,aid_CA_vs_mean_trn,aid_CA_rank_int_trn,aid_CA_rank_pct_trn,aid_OR_vs_mean_trn,aid_OR_rank_int_trn,aid_OR_rank_pct_trn,aid_CL2CA_tst,aid_CA2OR_tst,aid_CL2OR_tst,aid_CL_vs_mean_tst,aid_CL_rank_int_tst,aid_CL_rank_pct_tst,aid_CA_vs_mean_tst,aid_CA_rank_int_tst,aid_CA_rank_pct_tst,aid_OR_vs_mean_tst,aid_OR_rank_int_tst,aid_OR_rank_pct_tst,aid_CL2CA_tst_vs_trn,aid_CA2OR_tst_vs_trn,aid_CL2OR_tst_vs_trn,aid_CL_vs_mean_tst_vs_trn,aid_CL_rank_int_tst_vs_trn,aid_CL_rank_pct_tst_vs_trn,aid_CA_vs_mean_tst_vs_trn,aid_CA_rank_int_tst_vs_trn,aid_CA_rank_pct_tst_vs_trn,aid_OR_vs_mean_tst_vs_trn,aid_OR_rank_int_tst_vs_trn,aid_OR_rank_pct_tst_vs_trn,clicks_rating_test,carts_rating_test,orders_rating_test,clicks_rating_train,carts_rating_train,orders_rating_train,clicks_rating_full,carts_rating_full,orders_rating_full
0,0,0.000000,-1.000000,0.000000,0.506732,460602.0,0.248222,0.000000,1545169.5,0.832705,0.000000,1256772.0,0.677285,0.000000,-1.000000,0.000000,0.636198,199543.5,0.254687,0.000000,496451.0,0.633644,0.000000,413098.0,0.527256,-1.000000,-1.000000,-1.000000,1.255491,4.332233e-01,1.026043,-1.000000,3.212923e-01,0.760946,-1.000000,3.286976e-01,0.778485,252137,2000000,2000000,508078,2000000,2000000,484443,2000000,2000000
1,1,0.034483,0.000000,0.000000,0.408201,539951.5,0.290984,0.130085,1035279.0,0.557921,0.000000,1256772.0,0.677285,-1.000000,-1.000000,-1.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,-1.000000,-1.000000,-1.000000,2.000000e+06,2.000000,-1.000000,2.000000e+06,2.000000,-1.000000,2.000000e+06,2.000000,2000000,2000000,2000000,623157,1207610,2000000,627638,1210785,2000000
2,2,0.000000,-1.000000,0.000000,0.225214,813738.5,0.438530,0.000000,1545169.5,0.832705,0.000000,1256772.0,0.677285,-1.000000,-1.000000,-1.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,-1.000000,-1.000000,-1.000000,2.000000e+06,2.000000,-1.000000,2.000000e+06,2.000000,-1.000000,2.000000e+06,2.000000,2000000,2000000,2000000,950394,2000000,2000000,978377,2000000,2000000
3,3,0.122161,0.217949,0.026625,17.974922,13652.0,0.007357,20.293184,11903.0,0.006415,13.444350,23275.5,0.012543,0.147826,0.117647,0.017391,18.290694,4520.5,0.005770,26.793551,2626.0,0.003352,25.352252,6047.5,0.007719,1.210089,0.539792,0.653197,1.017567,3.311236e-01,0.784231,1.320323,2.206167e-01,0.522507,1.885718,2.598226e-01,0.615362,3461,2773,2553,10455,11920,22313,9857,10904,20955
4,4,0.065217,0.000000,0.000000,1.942474,156592.0,0.084389,1.170761,246221.0,0.132691,0.000000,1256772.0,0.677285,0.000000,-1.000000,0.000000,0.795248,162864.5,0.207872,0.000000,496451.0,0.633644,0.000000,413098.0,0.527256,0.000000,-1.000000,-1.000000,0.409399,1.040056e+00,2.463262,0.000000,2.016282e+00,4.775349,-1.000000,3.286976e-01,0.778485,134592,2000000,2000000,152737,287711,2000000,151982,273680,2000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855598,1855598,0.000000,-1.000000,0.000000,0.098531,1334178.5,0.719000,0.000000,1545169.5,0.832705,0.000000,1256772.0,0.677285,-1.000000,-1.000000,-1.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,-1.000000,-1.000000,-1.000000,2.000000e+06,2.000000,-1.000000,2.000000e+06,2.000000,-1.000000,2.000000e+06,2.000000,2000000,2000000,2000000,1535283,2000000,2000000,1559615,2000000,2000000
1855599,1855599,0.000000,-1.000000,0.000000,0.154835,1030696.0,0.555451,0.000000,1545169.5,0.832705,0.000000,1256772.0,0.677285,0.000000,-1.000000,0.000000,0.159050,605034.0,0.772233,0.000000,496451.0,0.633644,0.000000,413098.0,0.527256,-1.000000,-1.000000,-1.000000,1.027220,5.870150e-01,1.390282,-1.000000,3.212923e-01,0.760946,-1.000000,3.286976e-01,0.778485,487974,2000000,200

In [58]:
def get_multi_percent(df,sfx):
    for type_name, type_code in type_labels.items():
        t = df[df['type']==type_code].groupby(['aid','session'],as_index=False).agg({'ts':'count'})
        t['ts'] = (t['ts']>1).astype('int8')
        t = t.groupby(['aid'],as_index=False).agg({'ts':['count','sum']})
        t.columns = ['aid','users','multi_users']
        t[f'aid_multi_{type_name}_percent_{sfx}'] = t['multi_users']/t['users']
        del t['users'], t['multi_users']
        if type_code==0:
            out = t
        else:
            out = out.merge(t,on=['aid'],how='outer')
    del t
    gc_clear()
    return out

In [59]:
multi = get_multi_percent(test_df,"test")
multi = multi.merge(
    get_multi_percent(train_df,"train"),
    on=['aid'],
    how='outer'
)
multi = multi.merge(
    get_multi_percent(pd.concat([train_df,test_df]),"full"),
    on=['aid'],
    how='outer'
)


feats_df = feats_df.merge(multi, on=['aid'], how='outer')

del multi
gc_clear()

for sfx in ['test','train','full']:
    for act in ['clicks','carts','orders']:
        feats_df[f"aid_multi_{act}_percent_{sfx}"] = feats_df[f"aid_multi_{act}_percent_{sfx}"].fillna(-1)
        
        
feats_df

,aid,aid_CL2CA_trn,aid_CA2OR_trn,aid_CL2OR_trn,aid_CL_vs_mean_trn,aid_CL_rank_int_trn,aid_CL_rank_pct_trn,aid_CA_vs_mean_trn,aid_CA_rank_int_trn,aid_CA_rank_pct_trn,aid_OR_vs_mean_trn,aid_OR_rank_int_trn,aid_OR_rank_pct_trn,aid_CL2CA_tst,aid_CA2OR_tst,aid_CL2OR_tst,aid_CL_vs_mean_tst,aid_CL_rank_int_tst,aid_CL_rank_pct_tst,aid_CA_vs_mean_tst,aid_CA_rank_int_tst,aid_CA_rank_pct_tst,aid_OR_vs_mean_tst,aid_OR_rank_int_tst,aid_OR_rank_pct_tst,aid_CL2CA_tst_vs_trn,aid_CA2OR_tst_vs_trn,aid_CL2OR_tst_vs_trn,aid_CL_vs_mean_tst_vs_trn,aid_CL_rank_int_tst_vs_trn,aid_CL_rank_pct_tst_vs_trn,aid_CA_vs_mean_tst_vs_trn,aid_CA_rank_int_tst_vs_trn,aid_CA_rank_pct_tst_vs_trn,aid_OR_vs_mean_tst_vs_trn,aid_OR_rank_int_tst_vs_trn,aid_OR_rank_pct_tst_vs_trn,clicks_rating_test,carts_rating_test,orders_rating_test,clicks_rating_train,carts_rating_train,orders_rating_train,clicks_rating_full,carts_rating_full,orders_rating_full,aid_multi_clicks_percent_test,aid_multi_carts_percent_test,aid_multi_orders_percent_test,aid_multi_clicks_percent_train,aid_multi_carts_percent_train,aid_multi_orders_percent_train,aid_multi_clicks_percent_full,aid_multi_carts_percent_full,aid_multi_orders_percent_full
0,0,0.000000,-1.000000,0.000000,0.506732,460602.0,0.248222,0.000000,1545169.5,0.832705,0.000000,1256772.0,0.677285,0.000000,-1.000000,0.000000,0.636198,199543.5,0.254687,0.000000,496451.0,0.633644,0.000000,413098.0,0.527256,-1.000000,-1.000000,-1.000000,1.255491,4.332233e-01,1.026043,-1.000000,3.212923e-01,0.760946,-1.000000,3.286976e-01,0.778485,252137,2000000,2000000,508078,2000000,2000000,484443,2000000,2000000,0.000000,-1.000000,-1.0,0.194444,-1.000000,-1.000000,0.175000,-1.000000,-1.000000
1,1,0.034483,0.000000,0.000000,0.408201,539951.5,0.290984,0.130085,1035279.0,0.557921,0.000000,1256772.0,0.677285,-1.000000,-1.000000,-1.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,-1.000000,-1.000000,-1.000000,2.000000e+06,2.000000,-1.000000,2.000000e+06,2.000000,-1.000000,2.000000e+06,2.000000,2000000,2000000,2000000,623157,1207610,2000000,627638,1210785,2000000,-1.000000,-1.000000,-1.0,0.137931,0.000000,-1.000000,0.137931,0.000000,-1.000000
2,2,0.000000,-1.000000,0.000000,0.225214,813738.5,0.438530,0.000000,1545169.5,0.832705,0.000000,1256772.0,0.677285,-1.000000,-1.000000,-1.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,2000000.0,2.000000,-1.000000,-1.000000,-1.000000,-1.000000,2.000000e+06,2.000000,-1.000000,2.000000e+06,2.000000,-1.000000,2.000000e+06,2.000000,2000000,2000000,2000000,950394,2000000,2000000,978377,2000000,2000000,-1.000000,-1.000000,-1.0,0.062500,-1.000000,-1.000000,0.062500,-1.000000,-1.000000
3,3,0.122161,0.217949,0.026625,17.974922,13652.0,0.007357,20.293184,11903.0,0.006415,13.444350,23275.5,0.012543,0.147826,0.117647,0.017391,18.290694,4520.5,0.005770,26.793551,2626.0,0.003352,25.352252,6047.5,0.007719,1.210089,0.539792,0.653197,1.017567,3.311236e-01,0.784231,1.320323,2.206167e-01,0.522507,1.885718,2.598226e-01,0.615362,3461,2773,2553,10455,11920,22313,9857,10904,20955,0.295652,0.117647,0.5,0.398590,0.115385,0.117647,0.390086,0.115607,0.138889
4,4,0.065217,0.000000,0.000000,1.942474,156592.0,0.084389,1.170761,246221.0,0.132691,0.000000,1256772.0,0.677285,0.000000,-1.000000,0.000000,0.795248,162864.5,0.207872,0.000000,496451.0,0.633644,0.000000,413098.0,0.527256,0.000000,-1.000000,-1.000000,0.409399,1.040056e+00,2.463262,0.000000,2.016282e+00,4.775349,-1.000000,3.286976e-01,0.778485,134592,2000000,2000000,152737,287711,2000000,151982,273680,2000000,0.400000,-1.000000,-1.0,0.260870,0.000000,-1.000000,0.265734,0.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855598,1855598,0.000000,-1.000000,0.000000,0.098531,1334178.5,0.719000,0.000000,1545169.5,0.832705,0.000000,125677

In [60]:
def get_favourite_dows(df,sfx):
    df['dow'] = ((df['ts']%(7*24*60*60))//(24*60*60)).astype('int8')
    for type_name, type_code in type_labels.items():
        fav = df[
            df['type']==type_code
        ].groupby(
            ['aid','dow'],
            as_index=False
        ).agg(
            {'session': 'count'}
        ).sort_values(
            by=['aid','session'],
            ascending=[True,False]
        ).drop_duplicates(['aid'])
        del fav['session']
        fav.columns = ['aid',f'aid_{type_name}_favourite_dow_{sfx}']
        if type_code==0:
            out = fav
        else:
            out = out.merge(fav,on=['aid'],how='outer')
    del df['dow'], fav
    gc_clear()
    return out

In [61]:
fav = get_favourite_dows(test_df,"test")
fav = fav.merge(
    get_favourite_dows(train_df,"train"),
    on=['aid'],
    how='outer'
)
fav = fav.merge(
    get_favourite_dows(pd.concat([train_df,test_df]),"full"),
    on=['aid'],
    how='outer'
)

feats_df = feats_df.merge(fav, on=['aid'], how='outer')

del fav
gc_clear()

In [62]:
selected_feats = [
    'aid',
    'aid_CA2OR_trn',
    'aid_CA_rank_int_tst_vs_trn',
    'aid_CA_vs_mean_trn',
    'aid_CA_vs_mean_tst',
    'aid_CA_vs_mean_tst_vs_trn',
    'aid_CL2CA_trn',
    'aid_CL2CA_tst',
    'aid_CL2OR_trn',
    'aid_CL_rank_int_trn',
    'aid_CL_rank_int_tst_vs_trn',
    'aid_CL_rank_pct_tst_vs_trn',
    'aid_CL_vs_mean_trn',
    'aid_CL_vs_mean_tst_vs_trn',
    'aid_multi_clicks_percent_full',
    'aid_multi_orders_percent_train',
    'aid_clicks_favourite_dow_test',
    'carts_rating_full',
    'carts_rating_train',
    'clicks_rating_full',
    'clicks_rating_train',
    'orders_rating_full'
]

In [63]:
feats_df[selected_feats].to_parquet(f'feats/FE_aids_{MODE}.pqt',index=False)

In [64]:
feats_df[selected_feats]

,aid,aid_CA2OR_trn,aid_CA_rank_int_tst_vs_trn,aid_CA_vs_mean_trn,aid_CA_vs_mean_tst,aid_CA_vs_mean_tst_vs_trn,aid_CL2CA_trn,aid_CL2CA_tst,aid_CL2OR_trn,aid_CL_rank_int_trn,aid_CL_rank_int_tst_vs_trn,aid_CL_rank_pct_tst_vs_trn,aid_CL_vs_mean_trn,aid_CL_vs_mean_tst_vs_trn,aid_multi_clicks_percent_full,aid_multi_orders_percent_train,aid_clicks_favourite_dow_test,carts_rating_full,carts_rating_train,clicks_rating_full,clicks_rating_train,orders_rating_full
0,0,-1.000000,3.212923e-01,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,460602.0,4.332233e-01,1.026043,0.506732,1.255491,0.175000,-1.000000,3.0,2000000,2000000,484443,508078,2000000
1,1,0.000000,2.000000e+06,0.130085,-1.000000,-1.000000,0.034483,-1.000000,0.000000,539951.5,2.000000e+06,2.000000,0.408201,-1.000000,0.137931,-1.000000,NaN,1210785,1207610,627638,623157,2000000
2,2,-1.000000,2.000000e+06,0.000000,-1.000000,-1.000000,0.000000,-1.000000,0.000000,813738.5,2.000000e+06,2.000000,0.225214,-1.000000,0.062500,-1.000000,NaN,2000000,2000000,978377,950394,2000000
3,3,0.217949,2.206167e-01,20.293184,26.793551,1.320323,0.122161,0.147826,0.026625,13652.0,3.311236e-01,0.784231,17.974922,1.017567,0.390086,0.117647,4.0,10904,11920,9857,10455,20955
4,4,0.000000,2.016282e+00,1.170761,0.000000,0.000000,0.065217,0.000000,0.000000,156592.0,1.040056e+00,2.463262,1.942474,0.409399,0.265734,-1.000000,2.0,273680,287711,151982,152737,2000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855598,1855598,-1.000000,2.000000e+06,0.000000,-1.000000,-1.000000,0.000000,-1.000000,0.000000,1334178.5,2.000000e+06,2.000000,0.098531,-1.000000,0.000000,-1.000000,NaN,2000000,2000000,1559615,1535283,2000000
1855599,1855599,-1.000000,3.212923e-01,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,1030696.0,5.870150e-01,1.390282,0.154835,1.027220,0.166667,-1.000000,4.0,2000000,2000000,1083148,1092786,2000000
1855600,1855600,0.166667,4.215513e-01,0.780507,1.576091,2.019317,0.107143,1.000000,0.017857,329754.5,1.834801e+00,4.345531,0.788250,0.201775,0.228070,0.000000,0.0,355774,385181,326098,319179,653152
1855601,1855601,0.000000,2.000000e+06,0.910592,-1.000000,-1.000000,0.116667,-1.000000,0.000000,312364.0,2.000000e+06,2.000000,0.844554,-1.000000,0.233333,-1.000000,NaN,337946,351470,318330,311866,2000000


# AIDS-2-SESSIONS TRAIN

In [65]:
import pandas as pd, numpy as np
from tqdm.notebook import tqdm
import os, sys, pickle, glob, gc
from collections import Counter
import itertools
pd.set_option('display.max_columns', None)
from otto_utils import *

In [66]:
MODE = 0

In [67]:
type_labels = {'clicks':0, 'carts':1, 'orders':2}

if MODE==1:
    dataset = "otto-chunk-data-inparquet-format"
if MODE==0:
    dataset = "otto-validation"

def load_test():    
    dfs = []
    for e, chunk_file in enumerate(glob.glob(f'input/{dataset}/test_parquet/*')):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True)

test_df = load_test()
print('Test data has shape',test_df.shape)
print(f'{len(set(test_df.session))} unique sessions')

Test data has shape (7683577, 4)
1801251 unique sessions


In [68]:
test_df = test_df.sort_values(['session','ts'],ascending=[True,False]).reset_index(drop=True)
test_df['n'] = test_df.groupby('session').cumcount()

In [69]:
for type_name, type_code in type_labels.items():
    print(f"handling {type_name} ...")
    a = test_df[
        test_df.type==type_code
    ].groupby(
        ['session','aid'],
        as_index=False
    ).agg(
        {
            'ts' : ['count',max],
            'n' : min,
        }
    )
    a.columns = [
        'session',
        'aid',
        f'a2s_{type_name}_num',
        f'a2s_last_{type_name[:-1]}_ts',
        f'a2s_last_{type_name[:-1]}_index'
    ]
    
    if type_code==0:
        feats_df = a
    else:
        feats_df = feats_df.merge(a, on=['session','aid'], how='outer')

del a
gc_clear()

num_columns = ['a2s_clicks_num','a2s_carts_num','a2s_orders_num']
index_columns = ['a2s_last_click_index','a2s_last_cart_index','a2s_last_order_index']
ts_columns = ['a2s_last_click_ts','a2s_last_cart_ts','a2s_last_order_ts']


for col in num_columns:
    feats_df[col] = feats_df[col].fillna(0).astype('int16')
for col in index_columns:
    feats_df[col] = feats_df[col].fillna(999).astype('int16')
for col in ts_columns:
    feats_df[col] = feats_df[col].fillna(-1).astype('int')
        
feats_df['a2s_actions_num'] = feats_df[num_columns].sum(axis=1)
feats_df['a2s_last_action_index'] = feats_df[index_columns].min(axis=1)
feats_df['a2s_last_action_ts'] = feats_df[ts_columns].max(axis=1)

feats_df['a2s_best_action_type'] = 0
feats_df.loc[feats_df['a2s_carts_num'] > 0,'a2s_best_action_type'] = 1
feats_df.loc[feats_df['a2s_orders_num'] > 0,'a2s_best_action_type'] = 2
feats_df['a2s_best_action_type'] = feats_df['a2s_best_action_type'].astype('int8')

feats_df

handling clicks ...
handling carts ...
handling orders ...


,session,aid,a2s_clicks_num,a2s_last_click_ts,a2s_last_click_index,a2s_carts_num,a2s_last_cart_ts,a2s_last_cart_index,a2s_orders_num,a2s_last_order_ts,a2s_last_order_index,a2s_actions_num,a2s_last_action_index,a2s_last_action_ts,a2s_best_action_type
0,11098528,11830,1,1661119200,0,0,-1,999,0,-1,999,1,0,1661119200,0
1,11098529,1105029,1,1661119200,0,0,-1,999,0,-1,999,1,0,1661119200,0
2,11098530,264500,2,1661119288,4,0,-1,999,0,-1,999,2,4,1661119288,0
3,11098530,409236,3,1661120165,1,1,1661120532,0,0,-1,999,4,0,1661120532,1
4,11098531,396199,2,1661119383,11,0,-1,999,1,1661119746,3,3,3,1661119746,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5535985,12896465,1372417,0,-1,999,0,-1,999,1,1661722860,3,1,3,1661722860,2
5535986,12896465,1551275,0,-1,999,0,-1,999,1,1661722860,6,1,6,1661722860,2
5535987,12896768,1303029,0,-1,999,0,-1,999,1,1661723107,0,1,0,1661723107,2
5535988,12898765,73333,0,-1,999,0,-1,999,1,1661723432,0,1,0,1661723432,2


In [70]:
selected_feats = [
    'session','aid',
    'a2s_actions_num',
    'a2s_best_action_type',
    'a2s_carts_num',
    'a2s_clicks_num',
    'a2s_orders_num',
    'a2s_last_action_index',
    'a2s_last_cart_index',
    'a2s_last_click_index',
    'a2s_last_action_ts',
    'a2s_last_click_ts',
    'a2s_last_cart_ts',
    'a2s_last_order_ts'
]

In [71]:
feats_df[selected_feats].to_parquet(f"feats/FE_aids2sessions_{MODE}.pqt",index=False)

In [72]:
feats_df[selected_feats]

,session,aid,a2s_actions_num,a2s_best_action_type,a2s_carts_num,a2s_clicks_num,a2s_orders_num,a2s_last_action_index,a2s_last_cart_index,a2s_last_click_index,a2s_last_action_ts,a2s_last_click_ts,a2s_last_cart_ts,a2s_last_order_ts
0,11098528,11830,1,0,0,1,0,0,999,0,1661119200,1661119200,-1,-1
1,11098529,1105029,1,0,0,1,0,0,999,0,1661119200,1661119200,-1,-1
2,11098530,264500,2,0,0,2,0,4,999,4,1661119288,1661119288,-1,-1
3,11098530,409236,4,1,1,3,0,0,0,1,1661120532,1661120165,1661120532,-1
4,11098531,396199,3,2,0,2,1,3,999,11,1661119746,1661119383,-1,1661119746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5535985,12896465,1372417,1,2,0,0,1,3,999,999,1661722860,-1,-1,1661722860
5535986,12896465,1551275,1,2,0,0,1,6,999,999,1661722860,-1,-1,1661722860
5535987,12896768,1303029,1,2,0,0,1,0,999,999,1661723107,-1,-1,1661723107
5535988,12898765,73333,1,2,0,0,1,0,999,999,1661723432,-1,-1,1661723432


# AIDS-2-SESSIONS TEST

In [73]:
MODE = 1

In [74]:
type_labels = {'clicks':0, 'carts':1, 'orders':2}

if MODE==1:
    dataset = "otto-chunk-data-inparquet-format"
if MODE==0:
    dataset = "otto-validation"

def load_test():    
    dfs = []
    for e, chunk_file in enumerate(glob.glob(f'input/{dataset}/test_parquet/*')):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True)

test_df = load_test()
print('Test data has shape',test_df.shape)
print(f'{len(set(test_df.session))} unique sessions')

Test data has shape (6928123, 4)
1671803 unique sessions


In [75]:
test_df = test_df.sort_values(['session','ts'],ascending=[True,False]).reset_index(drop=True)
test_df['n'] = test_df.groupby('session').cumcount()

In [76]:
for type_name, type_code in type_labels.items():
    print(f"handling {type_name} ...")
    a = test_df[
        test_df.type==type_code
    ].groupby(
        ['session','aid'],
        as_index=False
    ).agg(
        {
            'ts' : ['count',max],
            'n' : min,
        }
    )
    a.columns = [
        'session',
        'aid',
        f'a2s_{type_name}_num',
        f'a2s_last_{type_name[:-1]}_ts',
        f'a2s_last_{type_name[:-1]}_index'
    ]
    
    if type_code==0:
        feats_df = a
    else:
        feats_df = feats_df.merge(a, on=['session','aid'], how='outer')

del a
gc_clear()

num_columns = ['a2s_clicks_num','a2s_carts_num','a2s_orders_num']
index_columns = ['a2s_last_click_index','a2s_last_cart_index','a2s_last_order_index']
ts_columns = ['a2s_last_click_ts','a2s_last_cart_ts','a2s_last_order_ts']


for col in num_columns:
    feats_df[col] = feats_df[col].fillna(0).astype('int16')
for col in index_columns:
    feats_df[col] = feats_df[col].fillna(999).astype('int16')
for col in ts_columns:
    feats_df[col] = feats_df[col].fillna(-1).astype('int')
        
feats_df['a2s_actions_num'] = feats_df[num_columns].sum(axis=1)
feats_df['a2s_last_action_index'] = feats_df[index_columns].min(axis=1)
feats_df['a2s_last_action_ts'] = feats_df[ts_columns].max(axis=1)

feats_df['a2s_best_action_type'] = 0
feats_df.loc[feats_df['a2s_carts_num'] > 0,'a2s_best_action_type'] = 1
feats_df.loc[feats_df['a2s_orders_num'] > 0,'a2s_best_action_type'] = 2
feats_df['a2s_best_action_type'] = feats_df['a2s_best_action_type'].astype('int8')

feats_df

handling clicks ...
handling carts ...
handling orders ...


,session,aid,a2s_clicks_num,a2s_last_click_ts,a2s_last_click_index,a2s_carts_num,a2s_last_cart_ts,a2s_last_cart_index,a2s_orders_num,a2s_last_order_ts,a2s_last_order_index,a2s_actions_num,a2s_last_action_index,a2s_last_action_ts,a2s_best_action_type
0,12899779,59625,1,1661724000,0,0,-1,999,0,-1,999,1,0,1661724000,0
1,12899780,582732,1,1661724058,3,0,-1,999,0,-1,999,1,3,1661724058,0
2,12899780,736515,1,1661724136,1,0,-1,999,0,-1,999,1,1,1661724136,0
3,12899780,973453,1,1661724109,2,0,-1,999,0,-1,999,1,2,1661724109,0
4,12899780,1142000,2,1661724155,0,0,-1,999,0,-1,999,2,0,1661724155,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5006552,14567528,172423,0,-1,999,0,-1,999,1,1662326425,8,1,8,1662326425,2
5006553,14567528,1708491,0,-1,999,0,-1,999,1,1662326425,9,1,9,1662326425,2
5006554,14568250,422075,0,-1,999,0,-1,999,1,1662326667,0,1,0,1662326667,2
5006555,14568250,471339,0,-1,999,0,-1,999,1,1662326667,1,1,1,1662326667,2


In [77]:
selected_feats = [
    'session','aid',
    'a2s_actions_num',
    'a2s_best_action_type',
    'a2s_carts_num',
    'a2s_clicks_num',
    'a2s_orders_num',
    'a2s_last_action_index',
    'a2s_last_cart_index',
    'a2s_last_click_index',
    'a2s_last_action_ts',
    'a2s_last_click_ts',
    'a2s_last_cart_ts',
    'a2s_last_order_ts'
]

In [78]:
feats_df[selected_feats].to_parquet(f"feats/FE_aids2sessions_{MODE}.pqt",index=False)

In [79]:
feats_df[selected_feats]

,session,aid,a2s_actions_num,a2s_best_action_type,a2s_carts_num,a2s_clicks_num,a2s_orders_num,a2s_last_action_index,a2s_last_cart_index,a2s_last_click_index,a2s_last_action_ts,a2s_last_click_ts,a2s_last_cart_ts,a2s_last_order_ts
0,12899779,59625,1,0,0,1,0,0,999,0,1661724000,1661724000,-1,-1
1,12899780,582732,1,0,0,1,0,3,999,3,1661724058,1661724058,-1,-1
2,12899780,736515,1,0,0,1,0,1,999,1,1661724136,1661724136,-1,-1
3,12899780,973453,1,0,0,1,0,2,999,2,1661724109,1661724109,-1,-1
4,12899780,1142000,2,0,0,2,0,0,999,0,1661724155,1661724155,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5006552,14567528,172423,1,2,0,0,1,8,999,999,1662326425,-1,-1,1662326425
5006553,14567528,1708491,1,2,0,0,1,9,999,999,1662326425,-1,-1,1662326425
5006554,14568250,422075,1,2,0,0,1,0,999,999,1662326667,-1,-1,1662326667
5006555,14568250,471339,1,2,0,0,1,1,999,999,1662326667,-1,-1,1662326667
